# Twitter Sentiment Analysis

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

from utils import *
from preprocessing import *

pd.options.mode.chained_assignment = None

## Load Data

In [2]:
data_path = '../data/'
pos_tweets_file = 'train_pos.txt'
neg_tweets_file = 'train_neg.txt'

In [3]:
pos_tweets = pd.read_table(data_path+pos_tweets_file, names=['tweet','sentiment'])
pos_tweets['sentiment'] = 'pos'
neg_tweets = pd.read_table(data_path+neg_tweets_file ,names=['tweet','sentiment'])
neg_tweets['sentiment'] = 'neg'

In [4]:
pos_tweets.head()

,tweet,sentiment
0,<user> i dunno justin read my mention or not ....,pos
1,"because your logic is so dumb , i won't even c...",pos
2,<user> just put casper in a box ! looved the...,pos
3,<user> <user> thanks sir > > don't trip lil ma...,pos
4,visiting my brother tmr is the bestest birthda...,pos


In [5]:
neg_tweets.head()

,tweet,sentiment
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,neg
1,glad i dot have taks tomorrow ! ! #thankful #s...,neg
2,1-3 vs celtics in the regular season = were fu...,neg
3,<user> i could actually kill that girl i'm so ...,neg
4,<user> <user> <user> i find that very hard to ...,neg


In [6]:
print('positive tweets shape: ',pos_tweets.shape)
print('negative tweets shape: ',neg_tweets.shape)

positive tweets shape:  (97902, 2)
negative tweets shape:  (99068, 2)


In [7]:
tweets = pd.concat([pos_tweets, neg_tweets], axis=0)

In [8]:
tweets.head()

,tweet,sentiment
0,<user> i dunno justin read my mention or not ....,pos
1,"because your logic is so dumb , i won't even c...",pos
2,<user> just put casper in a box ! looved the...,pos
3,<user> <user> thanks sir > > don't trip lil ma...,pos
4,visiting my brother tmr is the bestest birthda...,pos


In [9]:
tweets.tail()

,tweet,sentiment
99063,can't wait to fake tan tonight ! hate being pale,neg
99064,<user> darling i lost my internet connection ....,neg
99065,kanguru defender basic 4 gb usb 2.0 flash driv...,neg
99066,rizan is sad now,neg
99067,"no text back ? yea , he mad",neg


## Preprocessing

In [10]:
tweets['tweet'] = filter_user(tweets['tweet'])

In [11]:
tweets['tweet'] = filter_url(tweets['tweet'])

In [12]:
tweets['tweet'] = filter_hashtag(tweets['tweet'])

In [13]:
tweets['tokenized'] = tweets.apply(lambda tweet: tokenization(tweet['tweet']), axis=1)

In [14]:
# tweets['tagged'] = tweets.apply(lambda tweet: pos_tag(tweet['tweet']), axis=1)

In [15]:
tweets.head()

,tweet,sentiment,tokenized
0,@user i dunno justin read my mention or not . ...,pos,"[@user, i, dunno, justin, read, my, mention, o..."
1,"because your logic is so dumb , i won't even c...",pos,"[because, your, logic, is, so, dumb, ,, i, won..."
2,@user just put casper in a box ! looved the ...,pos,"[@user, just, put, casper, in, a, box, !, loov..."
3,@user @user thanks sir > > don't trip lil mama...,pos,"[@user, @user, thanks, sir, >, >, don't, trip,..."
4,visiting my brother tmr is the bestest birthda...,pos,"[visiting, my, brother, tmr, is, the, bestest,..."


## Classifier